<a href="https://colab.research.google.com/github/mobassir94/RSNA-Intracranial-Hemorrhage-Detection/blob/master/RSNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tensorflow-gpu==1.15

In [0]:
!pip install pydicom
import numpy as np
import pandas as pd
import pydicom
import os
import collections
import sys
import glob
import random
import cv2
import tensorflow as tf
import multiprocessing

from math import ceil, floor
from copy import deepcopy
from tqdm import tqdm
from imgaug import augmenters as iaa

import keras
import keras.backend as K
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model, load_model
from keras.utils import Sequence
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from google.colab import files

In [0]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
!pip install kaggle

In [0]:
!pip install -U -q kaggle
!mkdir  /root/.kaggle



In [0]:
files.upload()
!cp kaggle.json /root/.kaggle


Saving kaggle.json to kaggle.json


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download taindow/rsna-train-stage-1-images-png-224x

100% 12.9G/12.9G [05:06<00:00, 42.1MB/s]
100% 12.9G/12.9G [05:06<00:00, 45.2MB/s]


In [0]:
!kaggle datasets download taindow/rsna-test-stage-1-images-png-224x

100% 1.58G/1.58G [00:47<00:00, 41.7MB/s]
100% 1.58G/1.58G [00:47<00:00, 35.5MB/s]


In [0]:
!kaggle datasets download mobassir/rsnatraintest

 31% 5.00M/16.2M [00:00<00:00, 25.4MB/s]
100% 16.2M/16.2M [00:00<00:00, 54.1MB/s]


In [0]:
print(os.listdir('./'))

['.config', 'traintest', 'kaggle.json', 'rsna-train-stage-1-images-png-224x', 'rsnatraintest.zip', 'drive', 'rsna-test-stage-1-images-png-224x', 'sample_data']


In [0]:
!unzip rsnatraintest.zip  -d /content/traintest

Archive:  rsnatraintest.zip
replace /content/traintest/stage_1_sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: None


In [0]:
#!unzip rsna-train-stage-1-images-png-224x.zip  -d /content/rsna-train-stage-1-images-png-224x

!unzip -q rsna-train-stage-1-images-png-224x.zip  -d /content/rsna-train-stage-1-images-png-224x

unzip:  cannot find or open rsna-train-stage-1-images-png-224x.zip, rsna-train-stage-1-images-png-224x.zip.zip or rsna-train-stage-1-images-png-224x.zip.ZIP.


In [0]:
!unzip -q rsna-test-stage-1-images-png-224x.zip  -d /content/rsna-test-stage-1-images-png-224x

unzip:  cannot find or open rsna-test-stage-1-images-png-224x.zip, rsna-test-stage-1-images-png-224x.zip.zip or rsna-test-stage-1-images-png-224x.zip.ZIP.


In [0]:
os.remove('./rsna-train-stage-1-images-png-224x.zip')


FileNotFoundError: ignored

In [0]:
os.remove('./rsna-test-stage-1-images-png-224x.zip')

FileNotFoundError: ignored

In [0]:
os.remove('./rsnatraintest.zip')

In [0]:
dir_csv = './traintest'
dir_train_img = './rsna-train-stage-1-images-png-224x/stage_1_train_png_224x'
dir_test_img = './rsna-test-stage-1-images-png-224x/stage_1_test_png_224x'

In [0]:
# Install Modules from internet
!pip install efficientnet
!pip install iterative-stratification

In [0]:
# Import Custom Modules
import efficientnet.keras as efn 
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

In [0]:
# Seed
SEED = 12345
np.random.seed(SEED)
#tf.set_random_seed(SEED)

# Constants
TEST_SIZE = 0.15
HEIGHT = 224
WIDTH = 224
CHANNELS = 3
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

# Folders
#DATA_DIR = '/kaggle/input/rsna-intracranial-hemorrhage-detection/'
TEST_IMAGES_DIR = dir_test_img
TRAIN_IMAGES_DIR = dir_train_img

In [0]:
img = cv2.imread(TEST_IMAGES_DIR)
len(img)

TypeError: ignored

**reader**

In [0]:
def _read(path, SHAPE):
    dcm = cv2.imread(path)
    try:
        img = cv2.imread(path)
    except:
        img = np.zeros(SHAPE)
    return img

In [0]:
filepath = "/content/drive/My Drive/"

In [0]:
img = cv2.imread('/content/drive/My Drive/IMG_20170321_115850.jpg')
img.shape

(216, 720, 3)

In [0]:
# Image Augmentation
sometimes = lambda aug: iaa.Sometimes(0.25, aug)
augmentation = iaa.Sequential([ iaa.Fliplr(0.25),
                                iaa.Flipud(0.10),
                                sometimes(iaa.Crop(px=(0, 25), keep_size = True, sample_independently = False))   
                            ], random_order = True)       
        
# Generators
class TrainDataGenerator(keras.utils.Sequence):
    def __init__(self, dataset, labels, batch_size = 16, img_size = SHAPE, img_dir = TRAIN_IMAGES_DIR, augment = False, *args, **kwargs):
        self.dataset = dataset
        self.ids = dataset.index
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.augment = augment
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X, Y = self.__data_generation(indices)
        return X, Y

    def augmentor(self, image):
        augment_img = augmentation        
        image_aug = augment_img.augment_image(image)
        return image_aug

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))
        np.random.shuffle(self.indices)

    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
            image = _read(self.img_dir+ID+".png", self.img_size)########
            if self.augment:
                X[i,] = self.augmentor(image)
            else:
                X[i,] = image
            Y[i,] = self.labels.iloc[index].values        
        return X, Y
    
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, dataset, labels, batch_size = 16, img_size = SHAPE, img_dir = TEST_IMAGES_DIR, *args, **kwargs):
        self.dataset = dataset
        self.ids = dataset.index
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indices)
        return X

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))
    
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
            image = _read(self.img_dir+ID+".png", self.img_size)########
            X[i,] = image              
        return X

In [0]:
def read_testset(filename = dir_csv + "/stage_1_sample_submission.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    return df

def read_trainset(filename = dir_csv + "/stage_1_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    df = df.drop(index = duplicates_to_remove)
    df = df.reset_index(drop = True)    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    return df

# Read Train and Test Datasets
test_df = read_testset()
train_df = read_trainset()

In [0]:
# Oversampling
epidural_df = train_df[train_df.Label['epidural'] == 1]
train_oversample_df = pd.concat([train_df, epidural_df])
train_df = train_oversample_df

# Summary
print('Train Shape: {}'.format(train_df.shape))
print('Test Shape: {}'.format(test_df.shape))

Train Shape: (677019, 6)
Test Shape: (78545, 6)


In [0]:
def predictions(test_df, model):    
    test_preds = model.predict_generator(TestDataGenerator(test_df, None, 5, SHAPE, TEST_IMAGES_DIR), verbose = 1)
    return test_preds[:test_df.iloc[range(test_df.shape[0])].shape[0]]

def ModelCheckpointFull(model_name):
    return ModelCheckpoint(filepath + model_name, 
                            monitor = 'val_loss', 
                            verbose = 1, 
                            save_best_only = False, 
                            save_weights_only = True, 
                            mode = 'min', 
                            period = 1)

# Create Model
def create_model():
    #K.clear_session()
    
    base_model =  efn.EfficientNetB2(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = SHAPE)
    x = base_model.output
    x = Dropout(0.10)(x)
    y_pred = Dense(6, activation = 'sigmoid')(x)

    return Model(inputs = base_model.input, outputs = y_pred)


In [0]:
# Submission Placeholder
submission_predictions = []

# Multi Label Stratified Split stuff...
msss = MultilabelStratifiedShuffleSplit(n_splits = 10, test_size = TEST_SIZE, random_state = SEED)
X = train_df.index
Y = train_df.Label.values

# Get train and test index
msss_splits = next(msss.split(X, Y))
train_idx = msss_splits[0]
valid_idx = msss_splits[1]

In [0]:
# Loop through Folds of Multi Label Stratified Split
#for epoch, msss_splits in zip(range(0, 9), msss.split(X, Y)): 
#    # Get train and test index
#    train_idx = msss_splits[0]
#    valid_idx = msss_splits[1]
LR = 0.0011
for epoch in range(0, 7):
    print('=========== EPOCH {}'.format(epoch))

    # Shuffle Train data
    np.random.shuffle(train_idx)
    print(train_idx[:5])    
    print(valid_idx[:5])

    # Create Data Generators for Train and Valid
    data_generator_train = TrainDataGenerator(train_df.iloc[train_idx], 
                                                train_df.iloc[train_idx], 
                                                TRAIN_BATCH_SIZE, 
                                                SHAPE,
                                                augment = True)
    data_generator_val = TrainDataGenerator(train_df.iloc[valid_idx], 
                                            train_df.iloc[valid_idx], 
                                            VALID_BATCH_SIZE, 
                                            SHAPE,
                                            augment = False)

    # Create Model
    model = create_model()
    
    # Full Training Model
    for base_layer in model.layers[:-1]:
      base_layer.trainable = True
      TRAIN_STEPS = int(len(data_generator_train) / 6)
      # Load Model Weights
      if epoch != 0:
        a = filepath + "model.h5"
        model.load_weights(a)    

    model.compile(optimizer = Adam(LR), 
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])
    
    # Train Model
    model.fit_generator(generator = data_generator_train,
                        validation_data = data_generator_val,
                        steps_per_epoch = TRAIN_STEPS,
                        epochs = 1,
                        callbacks = [ModelCheckpointFull('model.h5')],
                        verbose = 1)
    
    # Starting with the 6th epoch we create predictions for the test set on each epoch
    if epoch >= 2:
        preds = predictions(test_df, model)
        submission_predictions.append(preds)

=========== EPOCH 0
[361997 260409 637585 604744 401268]
[ 0 17 27 30 44]











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
31940608/31936256 [==============================] - 2s 0us/step



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1


AttributeError: ignored

In [0]:
test_df.iloc[:, :] = np.average(submission_predictions, axis = 0, weights = [2**i for i in range(len(submission_predictions))])
test_df = test_df.stack().reset_index()
test_df.insert(loc = 0, column = 'ID', value = test_df['Image'].astype(str) + "_" + test_df['Diagnosis'])
test_df = test_df.drop(["Image", "Diagnosis"], axis=1)
test_df.to_csv('submission.csv', index = False)
print(test_df.head(12))

In [0]:

!cp submission.csv drive/My\ Drive/